In [1]:
!pip install pymysql
!pip install xgboost
import numpy as np
import pandas as pd
import datetime
import os
import warnings 
warnings.filterwarnings('ignore')
os.chdir('/home/ec2-user/SageMaker/daemon_branch/bonzo_ds_models/')
from models.ml_utils import *
from db_utils import *
import traceback

     |████████████████████████████████| 43 kB 458 kB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 173.5 MB 22 kB/s s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Inside init_config
Initialized logger path:  /home/ec2-user/SageMaker/daemon_branch/bonzo_ds_models


In [2]:
%%time
con = db_connection()
weekly_df = pd.read_sql_query("""select * from morphe_staging.fact_sales_ending_inventory_sku_by_week""", con=con)

connecting to db...
connected to db
CPU times: user 37.5 s, sys: 1.37 s, total: 38.8 s
Wall time: 38.9 s


In [3]:
%%time
daily_df = pd.read_sql_query("""select country,channel,sub_channel,gross_revenue,sales_date,sku from morphe_staging.sales_ending_inventory_sku_by_date""", con=con)
con.close()

CPU times: user 3min 26s, sys: 12 s, total: 3min 39s
Wall time: 3min 38s


In [79]:
daily_df1=daily_df.copy()
daily_df1['country']=daily_df1['country'].replace('AUS','AU').replace('CAN','CA').replace('CN','CA')

In [80]:
df_axes=pd.DataFrame()
df_axes['sales_date']=daily_df['sales_date']
df_axes['sales_date'] = pd.to_datetime(df_axes['sales_date'],errors='coerce')
df_axes.set_index('sales_date',inplace=True)
df_axes=df_axes.resample('m').count()
df_axes.reset_index(inplace=True)
df_axes

,sales_date
0,2017-01-31
1,2017-02-28
2,2017-03-31
3,2017-04-30
4,2017-05-31
5,2017-06-30
6,2017-07-31
7,2017-08-31
8,2017-09-30
9,2017-10-31


In [5]:
# daily_df1.groupby(by='sku').count()

## Monthly analysis of contribution of different subchannels towards total sales


In [81]:
daily_df1=daily_df1.groupby('channel').get_group('Ecomm')
daily_df1['sales_date'] = pd.to_datetime(daily_df1['sales_date'],errors='coerce')
daily_df1.set_index('sales_date',inplace=True)

In [30]:
daily_df1.sub_channel.unique()

array(['US', 'AU', 'CA', 'EU', 'JH', 'PL', 'UK', 'BadHabit', 'REM'],
      dtype=object)

In [31]:
# b=daily_df1.groupby('sub_channel').get_group('PL')
# b.groupby('sku').count()

In [82]:
sku_number=str(input("enter the sku number: "))
daily_df1=daily_df1.groupby('sku').get_group(sku_number)
x=daily_df1.sub_channel.unique()

enter the sku number: -1


In [33]:
# daily_df1

In [34]:
# x

In [35]:
# if 'PL' in x:
#     print('us present')
# else:
#     print('absent')

In [36]:
# daily_df1.sku.unique()

In [37]:
# daily_df1['sku'].dtype

In [83]:
if 'US' in x:
    df_US=daily_df1.groupby('sub_channel').get_group('US')
    df_US=df_US['gross_revenue'].resample('m').sum()
    df_US=df_US.to_frame()
    df_US.rename(columns={'gross_revenue':'US_sales'},inplace=True)
else:
    df_US=pd.DataFrame()

if 'PL' in x:
    df_PL=daily_df1.groupby('sub_channel').get_group('PL')
    df_PL=df_PL['gross_revenue'].resample('m').sum()
    df_PL=df_PL.to_frame()
    df_PL.rename(columns={'gross_revenue':'PL_sales'},inplace=True)
else:
    df_PL=pd.DataFrame()

if 'JH' in x:
    df_JH=daily_df1.groupby('sub_channel').get_group('JH')
    df_JH=df_JH['gross_revenue'].resample('m').sum()
    df_JH=df_JH.to_frame()
    df_JH.rename(columns={'gross_revenue':'JH_sales'},inplace=True)
else:
    df_JH=pd.DataFrame()

if 'BadHabit' in x:
    df_BadHabit=daily_df1.groupby('sub_channel').get_group('BadHabit')
    df_BadHabit=df_BadHabit['gross_revenue'].resample('m').sum()
    df_BadHabit=df_BadHabit.to_frame()
    df_BadHabit.rename(columns={'gross_revenue':'Badhabit_sales'},inplace=True)
else:
    df_BadHabit=pd.DataFrame()

if 'REM' in x:
    df_REM=daily_df1.groupby('sub_channel').get_group('REM')
    df_REM=df_REM['gross_revenue'].resample('m').sum()
    df_REM=df_REM.to_frame()
    df_REM.rename(columns={'gross_revenue':'REM_sales'},inplace=True)
else:
    df_REM=pd.DataFrame()


In [84]:
df_JH

""


In [85]:
if df_PL.empty:
    df_PL['PL_sales']=np.NAN
    df_PL['sales_date']=df_axes['sales_date']
#     df_PL['sales_date'] = pd.to_datetime(df_PL['sales_date'],errors='coerce')
    df_PL.set_index('sales_date',inplace=True)
#     df_PL=df_PL.resample('m')
    
else:pass

if df_US.empty:
    df_US['US_sales']=np.NAN
    df_US['sales_date']=df_axes['sales_date']
    df_US.set_index('sales_date',inplace=True)
#     df_US=df_US.resample('m')
else:pass

if df_BadHabit.empty:
    df_BadHabit['BadHabit_sales']=np.NAN
    df_BadHabit['sales_date']=df_axes['sales_date']
    df_BadHabit.set_index('sales_date',inplace=True)

else:pass

if df_REM.empty:
    df_REM['REM_sales']=np.NAN
    df_REM['sales_date']=df_axes['sales_date']
    df_REM.set_index('sales_date',inplace=True)
else:pass

if df_JH.empty:
    df_JH['JH_sales']=np.NAN
    df_JH['sales_date']=df_axes['sales_date']
    df_JH.set_index('sales_date',inplace=True)
else:pass

In [86]:
df_JH

,JH_sales
sales_date,
2017-01-31,NaN
2017-02-28,NaN
2017-03-31,NaN
2017-04-30,NaN
2017-05-31,NaN
2017-06-30,NaN
2017-07-31,NaN
2017-08-31,NaN
2017-09-30,NaN


In [65]:
# df_JH.reset_index(inplace=True)

In [66]:

# df_JH['sales_date'] = pd.to_datetime(df_JH['sales_date'],errors='coerce')
# df_JH.set_index('sales_date',inplace=True)


In [71]:
# df_JH=df_JH['JH_sales'].resample('m').count()

KeyError: 'Column not found: JH_sales'

In [70]:
# df_JH

In [42]:
# df_BadHabit['sales_date']=np.NAN


In [87]:
df_extended=pd.merge(df_BadHabit,df_REM,on='sales_date',how='outer')
df_extended=pd.merge(df_extended,df_JH,on='sales_date',how='outer')
df_extended=pd.merge(df_extended,df_PL,on='sales_date',how='outer')
df_extended=pd.merge(df_extended,df_US,on='sales_date',how='outer')

In [88]:
df_extended["sum"] = df_extended.sum(axis=1)
df_extended

,BadHabit_sales,REM_sales,JH_sales,PL_sales,US_sales,sum
sales_date,,,,,,
2017-01-31,NaN,NaN,NaN,NaN,46687.25,46687.25
2017-02-28,NaN,NaN,NaN,NaN,61376.71,61376.71
2017-03-31,NaN,NaN,NaN,NaN,75567.29,75567.29
2017-04-30,NaN,NaN,NaN,NaN,56910.48,56910.48
2017-05-31,NaN,NaN,NaN,NaN,61829.37,61829.37
2017-06-30,NaN,NaN,NaN,NaN,89405.20,89405.20
2017-07-31,NaN,NaN,NaN,NaN,121160.64,121160.64
2017-08-31,NaN,NaN,NaN,NaN,92561.58,92561.58
2017-09-30,NaN,NaN,NaN,NaN,74542.00,74542.00


In [89]:
df_percentage=df_extended.copy()

In [90]:
df_percentage.columns

Index(['BadHabit_sales', 'REM_sales', 'JH_sales', 'PL_sales', 'US_sales',
       'sum'],
      dtype='object')

In [91]:
if 'BadHabit_sales' in df_percentage.columns:
    df_percentage['BadHabit_sales']=(df_percentage['BadHabit_sales']/df_percentage['sum'])*100
if 'REM_sales' in df_percentage.columns:
    df_percentage['REM_sales']=(df_percentage['REM_sales']/df_percentage['sum'])*100
if 'JH_sales' in df_percentage.columns:
    df_percentage['JH_sales']=(df_percentage['JH_sales']/df_percentage['sum'])*100
if 'PL_sales' in df_percentage.columns:
    df_percentage['PL_sales']=(df_percentage['PL_sales']/df_percentage['sum'])*100
if 'US_sales' in df_percentage.columns:
    df_percentage['US_sales']=(df_percentage['US_sales']/df_percentage['sum'])*100
    
df_percentage['sum']=(df_percentage['sum']/df_percentage['sum'])*100
    


In [92]:
# df_percentage.replace(np.NaN,0,inplace=True)

In [93]:
display(f"Monthly sales for sku number {sku_number} is :", df_percentage)

'Monthly sales for sku number -1 is :'

,BadHabit_sales,REM_sales,JH_sales,PL_sales,US_sales,sum
sales_date,,,,,,
2017-01-31,NaN,NaN,NaN,NaN,100.0,100.0
2017-02-28,NaN,NaN,NaN,NaN,100.0,100.0
2017-03-31,NaN,NaN,NaN,NaN,100.0,100.0
2017-04-30,NaN,NaN,NaN,NaN,100.0,100.0
2017-05-31,NaN,NaN,NaN,NaN,100.0,100.0
2017-06-30,NaN,NaN,NaN,NaN,100.0,100.0
2017-07-31,NaN,NaN,NaN,NaN,100.0,100.0
2017-08-31,NaN,NaN,NaN,NaN,100.0,100.0
2017-09-30,NaN,NaN,NaN,NaN,100.0,100.0


## Yearly analysis of contribution of different subchannels towards total sales

In [ ]:
if 'US' in x:
    df_US=daily_df1.groupby('sub_channel').get_group('US')
    df_US=df_US['gross_revenue'].resample('y').sum()
    df_US=df_US.to_frame()
    df_US.rename(columns={'gross_revenue':'US_sales'},inplace=True)
else:
    df_US=pd.DataFrame()
if 'PL' in x:
    df_PL=daily_df1.groupby('sub_channel').get_group('PL')
    df_PL=df_PL['gross_revenue'].resample('y').sum()
    df_PL=df_PL.to_frame()
    df_PL.rename(columns={'gross_revenue':'PL_sales'},inplace=True)
else:
    df_PL=pd.DataFrame()
if 'JH' in x:
    df_JH=daily_df1.groupby('sub_channel').get_group('JH')
    df_JH=df_JH['gross_revenue'].resample('y').sum()
    df_JH=df_JH.to_frame()
    df_JH.rename(columns={'gross_revenue':'JH_sales'},inplace=True)
else:
    df_JH=pd.DataFrame()
if 'BadHabit' in x:
    df_BadHabit=daily_df1.groupby('sub_channel').get_group('BadHabit')
    df_BadHabit=df_BadHabit['gross_revenue'].resample('y').sum()
    df_BadHabit=df_BadHabit.to_frame()
    df_BadHabit.rename(columns={'gross_revenue':'BadHabit_sales'},inplace=True)
else:
    df_BadHabit=pd.DataFrame()
if 'REM' in x:
    df_REM=daily_df1.groupby('sub_channel').get_group('REM')
    df_REM=df_REM['gross_revenue'].resample('y').sum()
    df_REM=df_REM.to_frame()
    df_REM.rename(columns={'gross_revenue':'REM_sales'},inplace=True)
else:
    df_REM=pd.DataFrame()

In [ ]:
if df_PL.empty:
    df_PL['PL_sales']=np.NAN
else:pass

if df_US.empty:
    df_US['US_sales']=np.NAN
else:pass

if df_BadHabit.empty:
    df_BadHabit['BadHabit_sales']=np.NAN
else:pass

if df_REM.empty:
    df_REM['REM_sales']=np.NAN
else:pass

if df_JH.empty:
    df_JH['JH_sales']=np.NAN
else:pass

In [ ]:
try:
    df_extended=pd.merge(df_BadHabit,df_REM,on='sales_date',how='outer')
    df_extended=pd.merge(df_extended,df_JH,on='sales_date',how='outer')
    df_extended=pd.merge(df_extended,df_PL,on='sales_date',how='outer')
    df_extended=pd.merge(df_extended,df_US,on='sales_date',how='outer')
except:
    print(traceback.print.exc())

In [ ]:
df_extended["sum"] = df_extended.sum(axis=1)
df_extended

In [ ]:
df_percentage=df_extended.copy()

In [ ]:
if 'BadHabit_sales' in df_percentage.columns:
    df_percentage['BadHabit_sales']=(df_percentage['BadHabit_sales']/df_percentage['sum'])*100
if 'REM_sales' in df_percentage.columns:
    df_percentage['REM_sales']=(df_percentage['REM_sales']/df_percentage['sum'])*100
if 'JH_sales' in df_percentage.columns:
    df_percentage['JH_sales']=(df_percentage['JH_sales']/df_percentage['sum'])*100
if 'PL_sales' in df_percentage.columns:
    df_percentage['PL_sales']=(df_percentage['PL_sales']/df_percentage['sum'])*100
if 'US_sales' in df_percentage.columns:
    df_percentage['US_sales']=(df_percentage['US_sales']/df_percentage['sum'])*100
    
df_percentage['sum']=(df_percentage['sum']/df_percentage['sum'])*100

In [ ]:
display(f"Yearly sales for sku number {sku_number} is :", df_percentage)